In [1]:
"""Convert Senate speech data from 114th Congress to bag of words format.

The data is provided by [1]. Specifically, we use the `hein-daily` data. To 
run this script, make sure the relevant files are in 
`data/senate-speeches-114/raw/`. The files needed for this script are 
`speeches_114.txt`, `descr_114.txt`, and `114_SpeakerMap.txt`.

#### References
[1]: Gentzkow, Matthew, Jesse M. Shapiro, and Matt Taddy. Congressional Record 
     for the 43rd-114th Congresses: Parsed Speeches and Phrase Counts. Palo 
     Alto, CA: Stanford Libraries [distributor], 2018-01-16. 
     https://data.stanford.edu/congress_text
"""

import os
import setup_utils as utils

import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.feature_extraction.text import CountVectorizer
import re
import nltk

In [2]:
nltkstops = nltk.corpus.stopwords.words('english')

In [3]:
## Set params

stops = 'custom'

In [4]:
project_dir = os.path.abspath(
    os.path.join(os.path.dirname('.'), os.pardir)) 
data_dir = os.path.join(project_dir, "data\\paradigms\\raw")
save_dir = os.path.join(project_dir, "data\\paradigms\\clean\\"+stops)



In [5]:
def convert_markdown_to_plain_text(markdown_string):

    markdown_string = str(markdown_string)
    # Remove newlines
    plain_text = markdown_string.replace('\n', ' ')
    
    # Remove bold formatting (e.g., **text**)
    plain_text = re.sub(r'\*\*(.*?)\*\*', r'\1', plain_text)
    
    # Remove backslashes
    plain_text = plain_text.replace('\\', '')
    
    # Remove other markdown formatting if needed
    plain_text = plain_text.replace('\t', '')
    
    return plain_text

def fix_spaces(string):
    new_string = ' '.join(string.strip().split())
    return new_string

df = pd.read_csv(os.path.join(data_dir, 'paradigms.csv'), index_col=0)[['Judge Name', 'Paradigm',"Judge's CEDA rounds"]]
df = df[df["Judge's CEDA rounds"] > 5] #scare away tabroom ghosts
df.drop("Judge's CEDA rounds", axis=1, inplace=True)
df['Paradigm'] = df['Paradigm'].apply(lambda x: x.encode("utf-8").decode("utf-8").lower())
df['Paradigm'] = df['Paradigm'].apply(convert_markdown_to_plain_text)
df['Paradigm'] = df['Paradigm'].apply(fix_spaces)
df.sort_index(inplace=True)
df = df[~df['Judge Name'].isna()]
df = df[df['Paradigm'].str.len() > 5]
df.head()

,Judge Name,Paradigm
Judge ID,,
3,Aaron Hardy,it's been a number of years since i've been an...
1057,Shunta Jordan,just a brief update for the high school commun...
1088,Bill Smelko,please email me your speech documents. i have ...
1265,Maggie Berthiaume,maggie berthiaume woodward academy current coa...
1464,Bill Russell,bill russell judge philosophy overview- i love...


In [6]:
from nltk.tokenize import sent_tokenize, word_tokenize
df['Paradigm'] = df['Paradigm'].apply(lambda x: [word_tokenize(t) for t in sent_tokenize(x)])
df.head()


,Judge Name,Paradigm
Judge ID,,
3,Aaron Hardy,"[[it, 's, been, a, number, of, years, since, i..."
1057,Shunta Jordan,"[[just, a, brief, update, for, the, high, scho..."
1088,Bill Smelko,"[[please, email, me, your, speech, documents, ..."
1265,Maggie Berthiaume,"[[maggie, berthiaume, woodward, academy, curre..."
1464,Bill Russell,"[[bill, russell, judge, philosophy, overview-,..."


In [7]:
clone = df['Paradigm'].apply(lambda x: [word for sentence in x for word in sentence])
length_raw = len(set(item for sublist in clone for item in sublist))
length_raw

26987

In [8]:
# stupid phraseizer
from collections import defaultdict
from tqdm import tqdm

def count_word_occurrences(documents, T):
    delta = 2
    word_counts = defaultdict(int)
    word_following_counts = defaultdict(int)

    print('iterating through documents!')
    for document in tqdm(documents):
        for sentence in document:
            for i in range(len(sentence) - 1):
                word_i = sentence[i]
                word_j = sentence[i + 1]

                word_counts[word_i] += 1
                word_counts[word_j] += 1

                if word_i != word_j:
                    word_following_counts[(word_i, word_j)] += 1
    vocab_size = len(word_counts)
    result_dict = {}
    print('scoring terms')
    for (word_i, word_j), count_ij in tqdm(word_following_counts.items()):
        count_i = word_counts[word_i]
        count_j = word_counts[word_j]
        if count_i not in nltk.corpus.stopwords.words('english'):
            score = (count_ij - delta)*vocab_size / (count_i * count_j)
            if score > T:
                result_dict[(word_i, word_j)] = word_i + '_' + word_j

    return result_dict

In [9]:
def replacer(document, replacement_dict):
    new_doc = []
    for sentence in document:
        new_sentence = []
        i = 0
        while i < len(sentence) - 2:
            if (sentence[i], sentence[i+1]) in replacement_dict:
                new_sentence.append(replacement_dict[(sentence[i], sentence[i+1])])
                i += 2
            else:
                new_sentence.append(sentence[i])
                i += 1
        new_doc.append(new_sentence)
    return new_doc


In [10]:
two_grams = count_word_occurrences(df['Paradigm'], 4)


iterating through documents!


100%|██████████| 1262/1262 [00:01<00:00, 1235.75it/s]


scoring terms


100%|██████████| 241894/241894 [00:34<00:00, 6997.98it/s]


In [11]:
df['Paradigm'] = df['Paradigm'].apply(replacer, replacement_dict=two_grams)

In [12]:
clone = df['Paradigm'].apply(lambda x: [word for sentence in x for word in sentence])
length_2grams = len(set(item for sublist in clone for item in sublist))
length_2grams

26485

In [13]:
three_four_grams = count_word_occurrences(df['Paradigm'], 2)

iterating through documents!


100%|██████████| 1262/1262 [00:00<00:00, 1316.09it/s]


scoring terms


100%|██████████| 227315/227315 [00:31<00:00, 7256.13it/s]


In [14]:
df['Paradigm'] = df['Paradigm'].apply(replacer, replacement_dict=three_four_grams)

In [15]:
clone = df['Paradigm'].apply(lambda x: [word for sentence in x for word in sentence])
length_4grams = len(set(item for sublist in clone for item in sublist))
length_4grams

25550

In [16]:
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
def pos_tag_and_lemmatize(sentences):
    lemmatizer = WordNetLemmatizer()
    tagged_sentences = []
    for sentence in sentences: 
        tagged_words = pos_tag(sentence)    # Perform POS tagging
        lemmatized_words = [lemmatizer.lemmatize(word, pos=get_wordnet_pos(pos)) for word, pos in tagged_words]
        tagged_sentences.append(lemmatized_words)
    return tagged_sentences

df['Paradigm'] = df['Paradigm'].apply(pos_tag_and_lemmatize)

In [17]:
clone = df['Paradigm'].apply(lambda x: [word for sentence in x for word in sentence])
length_lemma = len(set(item for sublist in clone for item in sublist))
length_lemma

22526

In [18]:

speaker = np.array(df.index.values)
clone = df['Paradigm'].apply(lambda x: [word for sentence in x for word in sentence])
speeches = list(clone)

# Create mapping between names and IDs.
speaker_to_speaker_id = dict(
    [(y, x) for x, y in enumerate(speaker)])
author_indices = np.array(
    [speaker_to_speaker_id[s] for s in speaker])
author_map = np.array(list(speaker_to_speaker_id.keys()))

print(author_map.shape)
print(author_indices.shape)
print(speaker.shape)
len(set(speaker))

speeches = [' '.join(doc) for doc in speeches]


(1262,)
(1262,)
(1262,)


In [19]:
find_stops = True

In [20]:
len(nltkstops)

179

In [21]:
if find_stops:
    count_vectorizer = CountVectorizer(min_df=1,
                                   max_df=1., 
                                   ngram_range=(1, 1),
                                   stop_words=nltkstops)


    # Learn initial document term matrix. This is only initial because we use it to
    # identify words to exclude based on author counts.
    counts = count_vectorizer.fit_transform(speeches)

    vocabulary = np.array(
        [k for (k, v) in sorted(count_vectorizer.vocabulary_.items(), 
                                key=lambda kv: kv[1])])

In [22]:
counts_per_author = utils.bincount_2d(author_indices, counts.toarray())
min_authors_per_word = 8
author_counts_per_word = np.sum(counts_per_author > 0, axis=0)
ranking = np.flip(np.argsort(author_counts_per_word))
TF = counts.toarray().sum(axis=0)/counts.toarray().sum(axis=0).sum()

IDF = np.log((author_counts_per_word * (1/counts.shape[0]))**-1)


terms_per_author = counts.toarray().sum(axis=1)
terms_per_author = np.where(terms_per_author==0, 1, terms_per_author)
tfidfpart1 = counts.toarray()/terms_per_author[:, np.newaxis]


TFIDF = (tfidfpart1 * np.exp(IDF)).sum(axis=0) / np.where(author_counts_per_word==0, 1, author_counts_per_word).astype(float)

ppt = counts.toarray() / counts.toarray().sum(axis=0)
entropy = (np.multiply(ppt, -np.log(np.where(ppt==0, 1, ppt)))).sum(axis=0)


print(len(vocabulary))
print(counts.shape)
print(counts_per_author.shape)
print(author_counts_per_word.shape)
print(TF.shape)
print(IDF.shape)
#vocabulary[ranking][:20]
print(TFIDF.shape)
print(entropy.T.shape)


15974
(1262, 15974)
(1262, 15974)
(15974,)
(15974,)
(15974,)
(15974,)
(15974,)


In [23]:
vocab_df = pd.DataFrame({'words':vocabulary, 'TF':TF, 'IDF':IDF, 'TFIDF':TFIDF, 'entropy':entropy})
vocab_df.sort_values('TF', ascending=False)

,words,TF,IDF,TFIDF,entropy
3789,debate,0.027533,0.055389,0.034200,6.757441
1181,argument,0.019652,0.113139,0.026160,6.669567
14259,think,0.011251,0.324813,0.019743,6.401307
8713,make,0.010117,0.229702,0.016890,6.592842
712,aff,0.009906,0.472225,0.022234,6.283383
...,...,...,...,...,...
5472,fathom,0.000003,7.140453,4.949020,0.000000
5473,fatigue,0.000003,7.140453,2.848758,0.000000
5475,faultcx,0.000003,7.140453,2.390152,0.000000
5477,faux,0.000003,7.140453,4.459364,0.000000


In [24]:
top_TF = vocab_df.sort_values('TF', ascending=False)[['words','TF']].iloc[:1500].set_index('words')
bot_IDF = vocab_df.sort_values('IDF', ascending=True)[['words','IDF']].iloc[:1500].set_index('words')
bot_TFIDF = vocab_df.sort_values('TFIDF', ascending=True)[['words','TFIDF']].iloc[:1500].set_index('words')
top_entropy = vocab_df.sort_values('entropy', ascending=False)[['words','entropy']].iloc[:1500].set_index('words')

full = top_TF.join([bot_IDF,bot_TFIDF,top_entropy], how='outer')


In [25]:
full.shape

(1653, 4)

In [26]:
# full.to_excel('check_for_stops.xlsx')

In [27]:
new_stops = pd.read_excel('check_for_stops.xlsx')
new_stops = list(new_stops[new_stops['include']=='bad']['words'])

In [28]:
new_stops = list(set(nltkstops) | set(new_stops))

In [36]:

count_vectorizer = CountVectorizer(min_df=5,
                                max_df=1., 
                                ngram_range=(1, 1),
                                stop_words=new_stops)

counts = count_vectorizer.fit_transform(speeches)

vocabulary = np.array(
    [k for (k, v) in sorted(count_vectorizer.vocabulary_.items(), 
                            key=lambda kv: kv[1])])

In [44]:
author_indices = np.array(
    [speaker_to_speaker_id[s] for s in speaker])
# Remove speeches with not enough words.
existing_speeches = np.where(np.sum(counts, axis=1) > 1)[0]
counts_dense = counts[existing_speeches]
author_indices = author_indices[existing_speeches]

len(vocabulary)

[   0    1    2 ... 1259 1260 1261]
(1254, 4653)
(1262,)


4653

In [45]:
save_dir

'c:\\Users\\austin\\Documents\\python-projs\\uchicago\\research\\Thesis\\TBIP_testing\\data\\paradigms\\clean\\custom'

In [46]:
# Save data.
if not os.path.exists(save_dir):
  os.makedirs(save_dir)

# `counts.npz` is a [num_documents, num_words] sparse matrix containing the
# word counts for each document.
sparse.save_npz(os.path.join(save_dir, "counts.npz"),
                sparse.csr_matrix(counts_dense).astype(np.float32))
# `author_indices.npy` is a [num_documents] vector where each entry is an
# integer indicating the author of the corresponding document.
np.save(os.path.join(save_dir, "author_indices.npy"), author_indices)
# `vocabulary.txt` is a [num_words] vector where each entry is a string
# denoting the corresponding word in the vocabulary.
np.savetxt(os.path.join(save_dir, "vocabulary.txt"), vocabulary, fmt="%s")
# `author_map.txt` is a [num_authors] vector of strings providing the name of
# each author in the corpus.
np.savetxt(os.path.join(save_dir, "author_map.txt"), author_map, fmt="%s")
# `raw_documents.txt` contains all the documents we ended up using.
raw_documents = [document.replace("\n", ' ').replace("\r", ' ') 
                 for document in np.array(speeches)[existing_speeches]]
np.savetxt(os.path.join(save_dir, "raw_documents.txt"), 
           raw_documents, 
           fmt="%s")


In [47]:
len(speeches)

1262

In [48]:
len(existing_speeches)


1254

In [35]:

#og dataframe
df.to_csv(os.path.join(save_dir, 'id_name_para.csv'))
